<a href="https://colab.research.google.com/github/ShesterG/Sentiment_Tools_Evaluation/blob/master/scratch_work/sentiment_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
!pip3 install config

In [3]:
!pip install watson_developer_cloud

In [4]:
!pip install vaderSentiment

In [4]:
import sys
sys.path.append('../../')
import config

ModuleNotFoundError: ignored

In [5]:
import pandas as pd
import glob
import sys
sys.path.append('../../')
import config
import warnings
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report,f1_score
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1  import Features, EntitiesOptions, KeywordsOptions, SentimentOptions, CategoriesOptions

In [6]:
DATA_FILE_PATH = '/content/drive/MyDrive/NLPGh/'
CLEAN_DATA_FILE_NAME = 'Book8Clean.csv'
SAVE_FILE = True
TOKENIZED_DATA_FILE_NAME = 'Book8Tokenized.csv'

In [7]:
reviews = pd.read_csv(DATA_FILE_PATH + TOKENIZED_DATA_FILE_NAME)

In [8]:
pd.set_option('display.max_colwidth', None)

In [ ]:
#reviews = pd.concat([pd.read_csv(, names=['reviews', 'Sentiment]) for f in glob.glob('data/*.txt')], ignore_index = True)

In [20]:
reviews.head()

,Unnamed: 0,tweet,location_x,pretweet,candidate,LRPredScore,LRConfi,Sentiment,Unnamed: 8,tokens
0,2,"@NAkufoAddo H. E Nana please follow up on this FAKE NEWS, LIES and sue BBC for peddling false information. Hold alÃ¢â¬Â¦ https://t.co/SdwnMsxeUV",NaN,nana pleas follow thi fake new lie and sue bbc for peddl fals inform hold al,Nana,-3.0,1.0,0,NaN,"['nana', 'pleas', 'follow', 'thi', 'fake', 'new', 'lie', 'and', 'sue', 'bbc', 'for', 'peddl', 'fals', 'inform', 'hold', 'al']"
1,6,"@ChipsJhay @TruthFreema @AOuattara_PRCI @NAkufoAddo No, you haven't. He's using his freedom of speech as it is. He's covered.","Afamase Akotom, Ghana",no you havent he use hi freedom speech is he cover,Nana,NaN,NaN,0,NaN,"['no', 'you', 'havent', 'he', 'use', 'hi', 'freedom', 'speech', 'is', 'he', 'cover']"
2,7,@IssahGerard @HEgyiri @NAkufoAddo Please who is this and how is he your in law??,NaN,pleas who thi and how your law,Nana,-1.0,1.0,0,NaN,"['pleas', 'who', 'thi', 'and', 'how', 'your', 'law']"
3,17,@Kwame34580189 @bukolasaraki @NAkufoAddo Whatever.... Thought you had something there....,Nigeria,whatev thought you had someth there,Nana,NaN,NaN,0,NaN,"['whatev', 'thought', 'you', 'had', 'someth', 'there']"
4,18,@NAkufoAddo Remember that the good people of Gh have spoken therefore Soo be it,NaN,rememb that the good peopl have spoken therefor soo,Nana,1.0,1.0,0,NaN,"['rememb', 'that', 'the', 'good', 'peopl', 'have', 'spoken', 'therefor', 'soo']"


In [ ]:
reviews.shape

(2748, 2)

In [9]:
minimum_value = reviews['Sentiment'].value_counts().min()

In [10]:
def sample_minimum(reviews):
    return reviews.sample(minimum_value)

    

In [11]:
g = reviews.groupby('Sentiment')

In [12]:
new_df = g.apply(sample_minimum).reset_index(drop=True)
new_df.head()

,Unnamed: 0,tweet,location_x,pretweet,candidate,LRPredScore,LRConfi,Sentiment,Unnamed: 8,tokens
0,387,Will do. @NAkufoAddo is a tyrant! https://t.co/GG7FTRViwl,"London, England",will do tyrant,Nana,NaN,NaN,-1,NaN,"['will', 'do', 'tyrant']"
1,6801,"@Elvisyorm @NAkufoAddo Lack of wisdom, my people perish ampa. Like reading and stop making stupid assumptions and cÃ¢â¬Â¦ https://t.co/JIUwJAZ0Jj",NaN,lack wisdom peopl perish ampa like read and stop make stupid assumpt and,Nana,-2.0,0.5,-1,NaN,"['lack', 'wisdom', 'peopl', 'perish', 'ampa', 'like', 'read', 'and', 'stop', 'make', 'stupid', 'assumpt', 'and']"
2,76655,@JDMahama Which good work @JDMahama are you talking about. We no longer get short memories we remember you said eveÃ¢â¬Â¦ https://t.co/yQc0j9pcib,Ghana,which good work are you talk about longer get short memori rememb you said eve,John,2.0,1.0,-1,NaN,"['which', 'good', 'work', 'are', 'you', 'talk', 'about', 'longer', 'get', 'short', 'memori', 'rememb', 'you', 'said', 'eve']"
3,76192,@JDMahama Subvert what? Simple calculations is very difficult for you guys. The third shameful defeat awaits you agÃ¢â¬Â¦ https://t.co/TlnUybuDWx,Louis Trichardt,subvert what simpl calcul veri difficult for you guy the third shame defeat await you,John,-2.0,0.5,-1,NaN,"['subvert', 'what', 'simpl', 'calcul', 'veri', 'difficult', 'for', 'you', 'guy', 'the', 'third', 'shame', 'defeat', 'await', 'you']"
4,79392,We are done talking about who started free shs as we did start such policy with it being progressively free. We shaÃ¢â¬Â¦ https://t.co/7TFjm5Pvjg,NaN,are done talk about who start free sh did start such polici with be progress free sha,John,2.0,1.0,-1,NaN,"['are', 'done', 'talk', 'about', 'who', 'start', 'free', 'sh', 'did', 'start', 'such', 'polici', 'with', 'be', 'progress', 'free', 'sha']"


In [13]:
new_df['Sentiment'].value_counts()

 1    1000
-1    1000
Name: Sentiment, dtype: int64

### For VADERSentiment

In [14]:
analyser = SentimentIntensityAnalyzer()


In [22]:
def vader_sentiment(text):
    score = analyser.polarity_scores(text)
    if score['compound'] > 0:
        result = 1
    else:
        result = -1
    return result

### For TextBlob

In [23]:
def text_blob(text):
    score = TextBlob(text).sentiment.polarity
    if score > 0:
        result = 1
    else:
        result = -1
    return result

### For IBM-Watson

In [29]:
URL = "https://gateway-lon.watsonplatform.net/natural-language-understanding/api"
# API = "aNAfGJImJnR8NPTOu855xFilyu396r50FuddXeHeAjQd"

natural_language_understanding = NaturalLanguageUnderstandingV1(                                         
    version='2018-11-16',
    iam_apikey= config.API, # Use your API key here
    url= URL # paste the url here
    )

AttributeError: ignored

In [35]:
def Sentiment_score(input_text): 
    # Input text can be sentence, paragraph or document
    response = natural_language_understanding.analyze (text = input_text, features = Features(sentiment=SentimentOptions()), language='en').get_result()
    # From the response extract score which is between -1 to 1
    score = response.get('sentiment').get('document').get('score')
    if score > 0:
        result = 1
    else:
        result = 0
    return result

In [24]:
new_df['vader_sent'] = new_df['tokens'].apply(vader_sentiment)
new_df['textblob_sent'] = new_df['tokens'].apply(text_blob)


In [36]:
new_df['IBM_sent'] = new_df['tokens'].apply(Sentiment_score)

NameError: ignored

In [25]:
col = ['vader_sent','textblob_sent']
def pd_score(df,col,target):
    score = []
    for items in col:
        rate = accuracy_score(target, df[items])
        score.append(rate)
    scores = pd.Series(score, index=['VADER', 'TextBlob'])
    return scores
    

In [26]:
scores = pd_score(new_df,col,new_df['Sentiment'])

In [27]:
rate = accuracy_score(new_df['Sentiment'], new_df['vader_sent'])

In [28]:
rate

0.7695

In [29]:
rate = accuracy_score(new_df['Sentiment'], new_df['textblob_sent'])

In [30]:
rate

0.7095

In [ ]:
rate = accuracy_score(new_df['Sentiment'], new_df['IBM_sent'])

In [ ]:
rate

0.8784875183553598

In [ ]:
new_df.head()

,reviews,sentiments,vader_sent,textblob_sent,IBM_sent
0,Why are these sad little vegetables so overcoo...,0,0,0,0
1,The problem I have is that they charge $11.99 ...,0,0,1,0
2,I was mortified.,0,0,0,0
3,This is essentially a communications tool that...,0,0,0,0
4,Mediocre food.,0,0,0,0


In [ ]:
score = TextBlob(new_df['tokens'][1]).sentiment.polarity

In [ ]:
score

0.275

In [ ]:
score = analyser.polarity_scores(new_df['tokens'][1])

In [ ]:
score

{'neg': 0.164, 'neu': 0.738, 'pos': 0.097, 'compound': -0.25}

In [ ]:
print(scores)

VADER         0.814611
TextBlob      0.777166
IBM_Watson    0.878488
dtype: float64


In [ ]:
confusion_matrix(new_df['Sentiment'], new_df['IBM_sent'])

array([[1251,  111],
       [ 220, 1142]])

In [31]:
confusion_matrix(new_df['Sentiment'], new_df['vader_sent'])

array([[773, 227],
       [234, 766]])

In [32]:
confusion_matrix(new_df['Sentiment'], new_df['textblob_sent'])

array([[723, 277],
       [304, 696]])

In [45]:
print(classification_report(new_df['Sentiment'], new_df['IBM_sent']))

KeyError: ignored

In [33]:
print(classification_report(new_df['Sentiment'], new_df['vader_sent']))

              precision    recall  f1-score   support

          -1       0.77      0.77      0.77      1000
           1       0.77      0.77      0.77      1000

    accuracy                           0.77      2000
   macro avg       0.77      0.77      0.77      2000
weighted avg       0.77      0.77      0.77      2000



In [34]:
print(classification_report(new_df['Sentiment'], new_df['textblob_sent']))

              precision    recall  f1-score   support

          -1       0.70      0.72      0.71      1000
           1       0.72      0.70      0.71      1000

    accuracy                           0.71      2000
   macro avg       0.71      0.71      0.71      2000
weighted avg       0.71      0.71      0.71      2000



In [ ]:
new_df['mode'] = new_df[['vader_sent','textblob_sent', 'IBM_sent']].mode(axis=1)

In [ ]:
new_df.head(30)

,reviews,sentiments,vader_sent,textblob_sent,IBM_sent,mode
0,Why are these sad little vegetables so overcoo...,0,0,0,0,0
1,The problem I have is that they charge $11.99 ...,0,0,1,0,0
2,I was mortified.,0,0,0,0,0
3,This is essentially a communications tool that...,0,0,0,0,0
4,Mediocre food.,0,0,0,0,0
5,I asked multiple times for the wine list and a...,0,0,0,0,0
6,"I started this review with two stars, but I'm ...",0,0,0,1,0
7,the spaghetti is nothing special whatsoever.,0,0,1,0,0
8,So just beware.,0,0,0,0,0
9,These are the central themes of the film and t...,0,0,0,0,0


In [ ]:
confusion_matrix(new_df['Sentiment'], new_df['mode'])

array([[1203,  159],
       [ 239, 1123]])

In [ ]:
accuracy_score(new_df['Sentiment'], new_df['mode'])

0.8538913362701909

In [ ]:
print(classification_report(new_df['Sentiment'], new_df['mode']))


              precision    recall  f1-score   support

           0       0.83      0.88      0.86      1362
           1       0.88      0.82      0.85      1362

    accuracy                           0.85      2724
   macro avg       0.86      0.85      0.85      2724
weighted avg       0.86      0.85      0.85      2724

